[How to improve SQLite insert performance in Python 3.6? - Ask Codes](https://askcodes.net/questions/how-to-improve-sqlite-insert-performance-in-python-3-6-)

In [1]:
import sqlite3

conn = sqlite3.connect(':memory:')
# Print each statement, for demonstration purposes.
conn.set_trace_callback(print)

conn.execute("""CREATE TABLE tbl (col1 int, col2 int)""")

values = [(1, 2), (3, 4), (5, 6)]
conn.executemany("""INSERT INTO tbl (col1, col2) VALUES (?, ?)""", values)
conn.commit()

CREATE TABLE tbl (col1 int, col2 int)
BEGIN 
INSERT INTO tbl (col1, col2) VALUES (1, 2)
INSERT INTO tbl (col1, col2) VALUES (3, 4)
INSERT INTO tbl (col1, col2) VALUES (5, 6)
COMMIT


In [2]:
flattened_values = [x for tpl in values for x in tpl]
base_stmt = """INSERT INTO tbl (col1, col2) VALUES """
values_clause = ', '.join(['(?, ?)' for _ in range(len(flattened_values) // 2)])

stmt = base_stmt + values_clause
conn.execute(stmt, flattened_values)
conn.commit()

BEGIN 
INSERT INTO tbl (col1, col2) VALUES (1, 2), (3, 4), (5, 6)
COMMIT


In [3]:
flattened_values

[1, 2, 3, 4, 5, 6]

In [5]:
#import sqlite3

conn=sqlite3.connect(':memory:')
conn.set_trace_callback(print)
curs=conn.cursor()

#curs.execute("PRAGMA synchronous = OFF")


#Perform a series of UPDATE or INSERT operations.
conn.execute("""CREATE TABLE tbl (col1 int, col2 int)""")
curs.execute("BEGIN TRANSACTION")
values = [(1, 2), (3, 4), (5, 6)]
conn.executemany("""INSERT INTO tbl (col1, col2) VALUES (?, ?)""", values)

conn.commit()
conn.close()

CREATE TABLE tbl (col1 int, col2 int)
BEGIN TRANSACTION
INSERT INTO tbl (col1, col2) VALUES (1, 2)
INSERT INTO tbl (col1, col2) VALUES (3, 4)
INSERT INTO tbl (col1, col2) VALUES (5, 6)
COMMIT


**pragma**

In [14]:
conn   = sqlite3.connect('./test.db')
cursor = conn.cursor()

# Journal model
# DELETE | TRUNCATE | PERSIST | MEMORY | WAL | OFF    
cursor = cursor.execute("PRAGMA journal_mode;")
print(cursor.fetchall())


# synchronous
#     0 | OFF | 
#     1 | NORMAL | 
#     2 | FULL | 
#     3 | EXTRA
cursor = cursor.execute("PRAGMA synchronous;")
print(cursor.fetchall())
conn.close()

[('delete',)]
[(2,)]


The `DELETE journaling mode` is the normal behavior. 
In the DELETE mode, the rollback journal is deleted at the conclusion of each transaction. 
Indeed, the delete operation is the action that causes the transaction to commit. (See the document titled Atomic Commit In SQLite for additional detail.)

In [9]:
with sqlite3.connect(':memory:') as conn:
    cursor = conn.cursor()
    cursor = cursor.execute("PRAGMA pragma_name;")
    print(cursor.fetchall())

[]


https://docs.python.org/3/library/sqlite3.html
```
The INSERT statement implicitly opens a transaction, which needs to be committed before changes are saved in the database (see Transaction control for details). Call con.commit() on the connection object to commit the transaction:
```

In [1]:
import sqlite3

In [2]:
con = sqlite3.connect("tutorial.db", isolation_level = None)
con.execute('pragma journal_mode=wal;')
cur = con.cursor()

cur = cur.execute("DROP TABLE IF EXISTS Movie")
cur = cur.execute("CREATE TABLE movie(title, year, score)")

res = cur.execute("SELECT name FROM sqlite_master")
res.fetchone()

('movie',)

In [3]:
res = cur.execute("SELECT * FROM movie")
res.fetchall()

[]

In [4]:
#cur = cur.execute("BEGIN TRANSACTION")
cur = cur.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")

In [5]:
res = cur.execute("SELECT * FROM movie")
res.fetchall()

[('Monty Python and the Holy Grail', 1975, 8.2),
 ('And Now for Something Completely Different', 1971, 7.5)]

In [6]:
con.close()

In [5]:
res = cur.execute("SELECT name FROM sqlite_master WHERE name='spam'")
res.fetchone() is None

True

In [6]:
cur.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")

In [9]:
res = cur.execute("SELECT * FROM movie")

In [10]:
res.fetchall()

[('Monty Python and the Holy Grail', 1975, 8.2),
 ('And Now for Something Completely Different', 1971, 7.5)]

I don't even need commit to do it

In [11]:
con.close()